# Multi-dimensional knapsack

## Intro

This is a variant of the integer linear programming problem in which the goal is to maximize the value transported in a knapsack limited by capacity in more than one dimension by selecting the most suitable items.

## Model statement

$$
\begin{align}
    \text{max} \quad & \sum_{i \in I}{c_{i} x_{i}} \\
    \text{s.t.} \quad & \sum_{i \in I}{w_{d, i} x_{i}} \leq k_{d} & \forall ~d \in D\\
    & x_{i} \in \left \{ 0, 1 \right \} & \forall ~i \in I\\
\end{align}
$$

In [1]:
import numpy as np
from scipy.optimize import linprog
import pyomo.environ as pyo

## Using scipy

We must formulate this problem using the following matrix notation:

$$
\begin{align}
    \text{min} \quad & \boldsymbol{c}^T \boldsymbol{x} \\
    \text{s.t.} \quad & \boldsymbol{A}_{eq} \boldsymbol{x} = \boldsymbol{b}_{eq} \\
    & \boldsymbol{A}_{ub} \boldsymbol{x} \leq \boldsymbol{b}_{ub}\\
    & \boldsymbol{l} \leq \boldsymbol{x} \leq \boldsymbol{u}
\end{align}
$$

**Decision variables:**

$\boldsymbol{x}$ - Column vector with the amount of each item added to the knapsack

**Fixed parameters**

$\boldsymbol{c}$ - Negative value of the value of each item

$\boldsymbol{A}_{ub}$ - Matrix of inequality constraint terms:\
$a_{d, i}$ - Weight per unit of item $i$ on dimension $d$

$\boldsymbol{b}_{ub_{d}}$ - Knapsack capacity on dimension $d$

In [ ]:
# Random seed
np.random.seed(12)

# Weight associated with each item in each dimension (matrix form)
A_ub = np.array([
    np.random.normal(loc=5.0, scale=1.0, size=10).clip(0.5, 10.0),
    np.random.normal(loc=6.0, scale=2.0, size=10).clip(0.5, 10.0)
])

# Value associated with each item
c = -np.random.normal(loc=10.0, scale=1.0, size=10).clip(0.5, 20.0)

# knapsack capacity
b_ub = np.array([21.0, 22.0])

# Bounds (one quantity of each item)
bounds = [(0, 1),] * len(c)

In [3]:
sol = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

In [4]:
print(sol)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -44.81724486288822
              x: [ 1.000e+00  8.656e-01  0.000e+00  1.000e+00  0.000e+00
                   0.000e+00  0.000e+00  8.805e-01  1.000e+00  0.000e+00]
            nit: 2
          lower:  residual: [ 1.000e+00  8.656e-01  0.000e+00  1.000e+00
                              0.000e+00  0.000e+00  0.000e+00  8.805e-01
                              1.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  1.456e+00  0.000e+00
                              4.160e+00  5.104e-01  3.411e+00  0.000e+00
                              0.000e+00  5.386e+00]
          upper:  residual: [ 0.000e+00  1.344e-01  1.000e+00  0.000e+00
                              1.000e+00  1.000e+00  1.000e+00  1.195e-01
                              0.000e+00  1.000e+00]
                 marginals: [-1.181e+00  0.000e+00  0.000e+00 -4.314e+00
         

## Integrality constraints

Since version 1.9.0 scipy accepts integrality constraints as it now as a wrapper to a MILP solver.

$x_i \in \Z \quad \forall i \in I$

In [5]:
integrality_vector = np.full(c.shape[0], 1)

sol_int = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=integrality_vector)

In [6]:
print(sol_int)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -39.88187183116921
              x: [ 1.000e+00  0.000e+00  1.000e+00  1.000e+00  0.000e+00
                   0.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00]
            nit: -1
          lower:  residual: [ 1.000e+00  0.000e+00  1.000e+00  1.000e+00
                              0.000e+00  0.000e+00  0.000e+00  1.000e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [ 0.000e+00  1.000e+00  0.000e+00  0.000e+00
                              1.000e+00  1.000e+00  1.000e+00  0.000e+00
                              1.000e+00  1.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
        

## Using pyomo

In [7]:
model = pyo.ConcreteModel()

### Sets

In [8]:
model.I = pyo.Set(initialize=range(10))
model.D = pyo.Set(initialize=["weight", "volume"])

### Parameters

In [9]:
# Parameters of the knapsack
k = dict(zip(model.D, b_ub))
model.k = pyo.Param(model.D, initialize=k)

In [10]:
# Value of each item
model.c = pyo.Param(model.I, initialize=dict(zip(model.I, -c)))

# Fill a dictionary of weights based on A values
w = {}
for j, d in enumerate(model.D):
    for i, item in enumerate(model.I):
        w[d, item] = A_ub[j, i]

model.w = pyo.Param(model.D, model.I, initialize=w)

### Variables

In [11]:
model.x = pyo.Var(model.I, within=pyo.Binary)

### Constraints

In [12]:
def capacity_constraint(model, d):
    return sum(model.x[i] * model.w[d, i] for i in model.I) <= model.k[d]

model.capacity_constraint = pyo.Constraint(model.D, rule=capacity_constraint)

### Objective

In [13]:
def obj_function(model):
    return sum(model.x[i] * model.c[i] for i in model.I)
    
model.objective = pyo.Objective(rule=obj_function, sense=pyo.maximize)

### Solve

In [14]:
opt = pyo.SolverFactory('cbc')
opt.options['sec'] = 120

In [15]:
solution = opt.solve(model)
print(solution)


Problem: 
- Name: unknown
  Lower bound: 39.88187183
  Upper bound: 39.88187183
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 10
  Number of binary variables: 10
  Number of integer variables: 10
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.05
  Wallclock time: 0.05
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 3
  Error rc: 0
  Time: 0.10871052742004395
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [16]:
model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 39.88187183116921


In [17]:
model.x.display()

x : Size=10, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :   1.0 :     1 : False : False : Binary
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
      3 :     0 :   1.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :   0.0 :     1 : False : False : Binary
      6 :     0 :   0.0 :     1 : False : False : Binary
      7 :     0 :   1.0 :     1 : False : False : Binary
      8 :     0 :   0.0 :     1 : False : False : Binary
      9 :     0 :   0.0 :     1 : False : False : Binary


In [19]:
for i, item in enumerate(model.I):
    if abs(sol.x[i] - model.x[item].value) <= 1e-3:
        if sol.x[i] >= 1e-3:
            print(f"Item {item} was added in both situations")
        else:
            print(f"Item {item} was not added in any situation")
        
    elif sol.x[i] > model.x[item].value + 1e-3:
        if sol.x[i] == 1:
            print(f"Item {item} was completely added in relaxed problem only")
        else:
            xi = sol.x[i]
            print(f"Item {item} was partially added in relaxed problem only - value {xi:.2f}")
            
    elif sol.x[i] + 1e-3 < model.x[item].value:
        if sol.x[i] <= 1e-3:
            print(f"Item {item} was added only in integer problem")
        else:
            xi = sol.x[i]
            print(f"Item {item} was partially added in relaxed problem - value {xi:.2f} - but completely added in the integer version")

Item 0 was added in both situations
Item 1 was partially added in relaxed problem only - value 0.87
Item 2 was added only in integer problem
Item 3 was added in both situations
Item 4 was not added in any situation
Item 5 was not added in any situation
Item 6 was not added in any situation
Item 7 was partially added in relaxed problem - value 0.88 - but completely added in the integer version
Item 8 was completely added in relaxed problem only
Item 9 was not added in any situation


In [20]:
model.capacity_constraint.display()

capacity_constraint : Size=2
    Key    : Lower : Body               : Upper
    volume :  None :  20.73819050160827 :  22.0
    weight :  None : 18.894462023985934 :  21.0
